In [ ]:
!pip install pgmpy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pgmpy.estimators import HillClimbSearch, BicScore
from pgmpy.models import BayesianNetwork

In [ ]:
dfStat = pd.read_csv('LLCP2022_dropna.csv')
# Conta i valori 0
count_2 = (dfStat['_SEX'] == 2).sum()

# Conta i valori 1
count_1 = (dfStat['_SEX'] == 1).sum()

print(f"Numero di 2: {count_2}")
print(f"Numero di 1: {count_1}")

numero_elementi_totali = len(dfStat['_AGEG5YR'])
print('lunghezza _AGEG5YR', numero_elementi_totali)

conta_age = dfStat['_AGEG5YR'].value_counts()

print('VALORI _AGEG5YR', conta_age)

In [ ]:
df = pd.read_csv('df_bayesiana.csv')
df = df.drop(columns = ['Unnamed: 0'])
df = df.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
df.head(2)
df = df.fillna(-1)
print(df.isnull().sum())

In [ ]:
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df)
best_model = hc.estimate(max_iter = 1000)

# Stampa la struttura appresa
print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
model = BayesianNetwork(best_model.edges())

# Addestrare la rete bayesiana con i dati
from pgmpy.estimators import MaximumLikelihoodEstimator
model.fit(df, estimator=MaximumLikelihoodEstimator)

# Visualizzare le CPDs
for cpd in model.get_cpds():
    print(cpd)


# Definire la struttura della rete bayesiana
#model = BayesianNetwork([('A', 'B'), ('B', 'C'), ('A', 'D')])

# Ottenere il Markov Blanket di un nodo
#node = 'B'
#markov_blanket = model.get_markov_blanket(node)
#print(f"Markov Blanket del nodo {node}: {markov_blanket}")

In [ ]:
!pip install pgmpy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter

# Caricare il dataset
df = pd.read_csv('df_bayesiana.csv')
df = df.drop(columns = ['Unnamed: 0'])
#df = df.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
df = df.fillna(-1)
# Dividere il dataset in train e test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df)
best_model_structure = hc.estimate(max_iter = 10000, scoring_method=K2Score(df), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 # print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")




In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
def plot_bayesian_network(model):
    G = nx.DiGraph()

    # Aggiungi nodi e archi
    G.add_nodes_from(model.nodes())
    G.add_edges_from(model.edges())

    # Disegna la rete
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=500, node_color="skyblue", font_size=4, font_color="black", font_weight="bold", arrowsize=5)
    plt.savefig('rete.jpeg', format="jpeg")
    plt.show()

plot_bayesian_network(model)

In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Previsione sul test set
y_true = test_data['DIABTYPE'].values
y_pred = []

for index, row in test_data.iterrows():
    evidence = row.drop('DIABTYPE').to_dict()
    
    # Debug: stampa l'evidenza per ogni riga
    print(f'Evidenza per index {index}: {evidence}')
    
    try:
        prediction = inference.map_query(variables=['DIABTYPE'], evidence=evidence)
        y_pred.append(prediction['DIABTYPE'])
    except Exception as e:
        print(f'Errore al processing dell\'indice {index}: {e}')
        y_pred.append(None)

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
df2 = pd.read_csv('df_bayesiana2.csv')
df2 = df2.drop(columns = ['Unnamed: 0'])
#df2 = df2.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
df2.head(2)
df2 = df2.fillna(-1)
print(df2.isnull().sum())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter

# Caricare il dataset
df2 = pd.read_csv('df_bayesiana2.csv')
df2 = df2.drop(columns = ['Unnamed: 0'])
df2 = df2.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
df2 = df2.fillna(-1)
# Dividere il dataset in train e test
train_data, test_data = train_test_split(df2, test_size=0.2, random_state=42)

# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df2)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(df2), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 # print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")




In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Previsione sul test set
y_true = test_data['DIABETE4'].values
y_pred = []

for index, row in test_data.iterrows():
    evidence = row.drop('DIABETE4').to_dict()
    
    # Debug: stampa l'evidenza per ogni riga
    print(f'Evidenza per index {index}: {evidence}')
    
    try:
        prediction = inference.map_query(variables=['DIABETE4'], evidence=evidence)
        y_pred.append(prediction['DIABETE4'])
    except Exception as e:
        print(f'Errore al processing dell\'indice {index}: {e}')
        y_pred.append(None)

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter
import dask.dataframe as dd
from sklearn.preprocessing import StandardScaler

# Caricare il dataset
df3 = pd.read_csv('df_bayesiana3.csv')
print(df3)
df3 = df3.drop(columns = ['Unnamed: 0'])

df3.replace({7: -1, 9: -1, 77 : -1, 99 : -1, 777: -1, 999: -1, 7777: -1, 9999: -1 ,'nan': -1}, inplace=True)
df3 = df3.astype({col: 'float32' for col in df3.select_dtypes(include='float64').columns})
df3 = df3.astype('float32')
#scaler = StandardScaler()
df3_sample =  df3

#print(df3_sample)
print(df3.dtypes)
#df3 = df3.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
#df3 = df3.fillna(-1)
# Dividere il dataset in train e test
train_data, test_data = train_test_split(df3_sample, test_size=0.3, random_state=42)

# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df3_sample)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(df3_sample))
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 # print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter

# Caricare il dataset
df3 = pd.read_csv('df_bayesiana3.csv')
print(df3)
df3 = df3.drop(columns = ['Unnamed: 0'])

df3.replace({7: -1, 9: -1, 77 : -1, 99 : -1, 777: -1, 999: -1, 7777: -1, 9999: -1 ,'nan': -1}, inplace=True)
df3.replace(-1.0, np.nan, inplace=True)
# Contare i NaN per colonna
nan_counts = df3.isna().sum()

# Determinare la soglia
soglia = 120000  # Modificare questo valore in base alla soglia desiderata

# Se il numero di valori mancanti in una colonna è maggiore della soglia, eliminare la colonna
df_cleaned = df3.drop(columns=nan_counts[nan_counts > soglia].index)
df_cleaned = df_cleaned.astype({col: 'float32' for col in df_cleaned.select_dtypes(include='float64').columns})
print(df_cleaned.dtypes)
print(df_cleaned)
#df3 = df3.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
#df3 = df3.fillna(-1)
# Dividere il dataset in train e test
train_data, test_data = train_test_split(df_cleaned, test_size=0.3, random_state=42)
# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df_cleaned)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(df_cleaned))
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 # print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")




In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Previsione sul test set
y_true = test_data['DIABETE4'].values
y_pred = []

for index, row in test_data.iterrows():
    evidence = row.drop('DIABETE4').to_dict()
    
    # Debug: stampa l'evidenza per ogni riga
    print(f'Evidenza per index {index}: {evidence}')
    
    try:
        prediction = inference.map_query(variables=['DIABETE4'], evidence=evidence)
        
        # Debug: stampa il risultato della previsione
        print(f'Prediction per index {index}: {prediction}')
        
        # Verifica il tipo di prediction
        if isinstance(prediction, dict) and 'DIABETE4' in prediction:
            y_pred.append(prediction['DIABETE4'])
        else:
            print(f'Predizione non valida per index {index}: {prediction}')
            y_pred.append(None)
            
    except Exception as e:
        print(f'Errore al processing dell\'indice {index}: {e}')
        y_pred.append(None)

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
import numpy as np
df = pd.read_sas('LLCP2022.XPT', format='xport')
df_bayesiana = df[['_BMI5', 'SMOKE100', 'CVDSTRK3', '_MICHD', '_TOTINDA', 'MEDCOST1', 'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK', 'LANDSEX1', '_AGEG5YR', 'EDUCA', 'INCOME3','DIABETE4', 'DIABTYPE']]
print(df_bayesiana)
rows_with_all_nan = df_bayesiana.isna().all(axis='columns')

if rows_with_all_nan.any():
    print("Sì, ci sono righe con tutti i valori NaN.")
    print("Indici delle righe con tutti i valori NaN:")
    print(rows_with_all_nan[rows_with_all_nan].index)
else:
    print("Non ci sono righe con tutti i valori NaN.")

columns_with_all_nan = df_bayesiana.columns[df_bayesiana.isna().all()]

if len(columns_with_all_nan) == 0:
    print("Non ci sono colonne con tutti i valori NaN.")
else:
    print("Colonne con tutti i valori NaN:")
    print(columns_with_all_nan)

df_bayesiana = df_bayesiana.drop(columns=columns_with_all_nan)
print(df_bayesiana.head(2))
df_bayesiana.replace({7: -1, 9: -1, 77 : -1, 99 : -1, 777: -1, 999: -1, 7777: -1, 9999: -1 ,'nan': -1}, inplace=True)
# Poiché 'NaN' potrebbe essere interpretato come valore NaN e non come stringa 'NaN', sostituiamo anche i valori np.nan
df_bayesiana.replace(np.nan, -1, inplace=True)
df_bayesiana.to_csv('df_bayesiana_paper.csv')

In [ ]:
print(df_bayesiana)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter

# Caricare il dataset
dfP = df_bayesiana
dfP = dfP.astype(np.float32)
def reduce_memory_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memoria usata dal DataFrame: {start_mem:.2f} MB -> {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% ridotta)')
    return df
# Applicazione della funzione al dataset
dfP = reduce_memory_usage(dfP)

#dfP = dfP.drop(columns = ['DIABETE4'])

#df3 = df3.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
#dfP = dfP.fillna(-1)
#dfP = dfP.drop(columns = ['Unnamed: 0'])
# Dividere il dataset in train e test
train_data, test_data = train_test_split(dfP, test_size=0.2, random_state=42)

# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(dfP)
best_model_structure = hc.estimate(max_iter = 1000000, scoring_method=K2Score(dfP))
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 # print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")




In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Previsione sul test set
y_true = test_data['DIABETE4'].values
y_pred = []

for index, row in test_data.iterrows():
    evidence = row.drop('DIABETE4').to_dict()
    
    # Debug: stampa l'evidenza per ogni riga
    #print(f'Evidenza per index {index}: {evidence}')
    
    try:
        prediction = inference.map_query(variables=['DIABETE4'], evidence=evidence)
        y_pred.append(prediction['DIABETE4'])
    except Exception as e:
        print(f'Errore al processing dell\'indice {index}: {e}')
        y_pred.append(None)

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
print(model)

In [ ]:
best_model_structure.edges()

In [ ]:
def reduce_memory_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memoria usata dal DataFrame: {start_mem:.2f} MB -> {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% ridotta)')
    return df

# Applicazione della funzione al dataset
train_data = reduce_memory_usage(train_data)


In [ ]:
def train_in_batches(model, data, batch_size):
    n_batches = int(np.ceil(len(data) / batch_size))
    metrics = []
    for i in range(n_batches):
        batch_data = data[i * batch_size: (i + 1) * batch_size]
        model.fit(batch_data, estimator=MaximumLikelihoodEstimator)
        # Calcolo della log-likelihood dopo ogni batch
        bic = BicScore(data)
        log_likelihood = bic.score(model)
        metrics.append(log_likelihood)
        print(f'Batch {i+1}/{n_batches} addestrato, Log-Likelihood: {log_likelihood}')
    return model, metrics

# Imposta la dimensione del batch
batch_size = 64

# Addestra il modello in batch
trained_model = train_in_batches(model, train_data, batch_size)
print("Metriche finali per ciascun batch:")
for i, metric in enumerate(training_metrics):
    print(f'Batch {i+1}: Log-Likelihood = {metric}')

In [ ]:
!pip install --upgrade pgmpy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BicScore

df = pd.read_sas('LLCP2022.XPT', format='xport')
df_bayesiana = df[['_AGEG5YR', 'DIABEDU1', 'DIABAGE4', 'DIABTYPE', 'INSULIN1',  '_BMI5', 'SMOKE100', 'CVDSTRK3', '_MICHD', '_TOTINDA', 'MEDCOST1', 'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK', 'LANDSEX1', '_AGEG5YR', 'INCOME3',  '_STSTR', '_EDUCAG', 'DIABEYE1', 'HEIGHT3', 'SLEPTIM1', 'WEIGHT2', 'COVIDINT', 'PFPPRVN4', 'MARJEAT', 'HPVADSHT', 'BLDSTFIT', 'CDHELP', 'DIABETE4']]
print(df_bayesiana)

df_bayesiana = df_bayesiana.dropna(axis=1, how='all')
print("axis=0",df_bayesiana)
df_bayesiana = df_bayesiana.dropna(axis=0, how='all')
print("axis=1",df_bayesiana)
#df_bayesiana = df_bayesiana.fillna(-1)
#print("fillna",df_bayesiana)
df_bayesiana.replace({7: -1, 9: -1, 77: -1, 99: -1, 777: -1, 999: -1, 7777: -1, 9999: -1, np.nan: -1}, inplace=True)
# Poiché 'NaN' potrebbe essere interpretato come valore NaN e non come stringa 'NaN', sostituiamo anche i valori np.nan
print("df finale: ",df_bayesiana)
print(df_bayesiana.dtypes)

#df_bayesiana.to_csv('df_bayesiana_paper.csv')




df finale:          _AGEG5YR  DIABEDU1  DIABAGE4  DIABTYPE  INSULIN1   _BMI5  SMOKE100  \
0           13.0      -1.0      80.0      -1.0      -1.0    -1.0       2.0   
1           13.0      -1.0      -1.0      -1.0      -1.0  2657.0       2.0   
2            8.0      -1.0      -1.0      -1.0      -1.0  2561.0       2.0   
3           14.0      -1.0      -1.0      -1.0      -1.0  2330.0       1.0   
4            5.0      -1.0      -1.0      -1.0      -1.0  2177.0       2.0   
...          ...       ...       ...       ...       ...     ...       ...   
445127       1.0      -1.0      -1.0      -1.0      -1.0  2563.0       2.0   
445128      -1.0      -1.0      -1.0      -1.0      -1.0  2866.0       2.0   
445129      10.0      -1.0      -1.0      -1.0      -1.0  1723.0       1.0   
445130      11.0      -1.0      -1.0      -1.0      -1.0  3255.0       2.0   
445131       5.0      -1.0      -1.0      -1.0      -1.0  2260.0       1.0   

        CVDSTRK3  _MICHD  _TOTINDA  ...  HEIGHT3  S

In [ ]:
train_data, test_data = train_test_split(df_bayesiana, test_size=0.2, random_state=42)

# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df_bayesiana)
best_model_structure = hc.estimate(max_iter = 1000000, scoring_method=K2Score(df_bayesiana))
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 # print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")

inference = VariableElimination(model)

# Previsione sul test set
y_true = test_data['DIABETE4'].values
y_pred = []

for index, row in test_data.iterrows():
    evidence = row.drop('DIABETE4').to_dict()
    
    # Debug: stampa l'evidenza per ogni riga
    #print(f'Evidenza per index {index}: {evidence}')
    
    try:
        prediction = inference.map_query(variables=['DIABETE4'], evidence=evidence)
        y_pred.append(prediction['DIABETE4'])
    except Exception as e:
        print(f'Errore al processing dell\'indice {index}: {e}')
        y_pred.append(None)

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')